In [4]:

import firebase_admin
from firebase_admin import credentials, firestore


service_account_path = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/atopile-880ca67acfe2.json"

if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

db = firestore.client()
_components_db = db.collection(
    "components"
)  

In [27]:
import json


def get_resistor(req_json, firestore_client):
    # Get the component requirements
    min_value = req_json.get("min_value")
    max_value = req_json.get("max_value")
    package = req_json.get("package")

    # Query for resistors in the Firestore collection
    query = firestore_client.collection("components").where("type", "==", "Resistor")

    # Execute the query and get results
    results = query.stream()
    components = [doc.to_dict() for doc in results]
    print(len(components))

    #Strip leading 'R' from the package
    package = package[1:]

    # Filter the components by the requirements
    filtered_components = []
    for component in components:
        if min_value <= component["min_value"] and max_value >= component["max_value"] and component["Package"] == package:
            filtered_components.append(component)


    # Remove any that are out of stock
    filtered_components = [component for component in filtered_components if component["Stock"] > 100]

    # Find the cheapest component
    best_resistor = min(filtered_components, key=lambda component: component["Price (USD)"])

    # Send back a JSON response with the cheapest resistor
    return json.dumps(best_resistor)

In [31]:
payload = {
    "type": "Resistor",
    "min_value": 900,
    "max_value": 1100,
    "package": "R0805"
}

get_resistor(payload, db)

1489


'{"value": 1.0, "type": "Resistor", "basic-extended": "Basic", "Description": "CHIP RESISTOR - SURFACE MOUNT 1KOHMS \\u00b11% 1/8W 0805 ROHS", "Price (USD)": 0.0027, "Stock": 1957712.0, "code": NaN, "Datasheet": "https://datasheet.lcsc.com/szlcsc/Uniroyal-Elec-0805W8F1001T5E_C17513.pdf", "Type": "CHIP RESISTOR - SURFACE MOUNT", "Package": "0805", "voltage": NaN, "min_value": 990.0, "MFR.Part #": "0805W8F1001T5E", "tolerance": 1.0, "value_in_ohms": NaN, "unit": "K\\u03a9", "LCSC Part #": "C17513", "Comment": NaN, "value_in_farads": NaN, "Category": NaN, "dielectric": NaN, "max_value": 1010.0}'

In [52]:
def get_mosfet(req_json, firestore_client):
# example json:
# {
#     "polarity": "N Channel",
#     "drain_source_voltage_V": 5,
#     "current_A": 0.1,
#     "on_resistance_ohms": "0.1",
#     "power_dissipation_W": "23"
# }
    query = firestore_client.collection("mosfets").where("type", "==", req_json["polarity"])
    results = query.stream()
    components = [doc.to_dict() for doc in results]
    # print(len(components))
    print(components[0])

    filtered_components = []
    for component in components:
        if component["drain_source_voltage_V"] >= req_json["drain_source_voltage_V"] and component["current_A"] >= req_json["current_A"]:# and component["on_resistance_ohms"] <= req_json["on_resistance_ohms"] and component["power_dissipation_W"] >= req_json["power_dissipation_W"]:
            filtered_components.append(component)
    print(len(filtered_components))
    best_mosfet = min(filtered_components, key=lambda component: component["Price"][0]["price"])



In [53]:
#

query = {
    "polarity": "N Channel",
    "drain_source_voltage_V": "30",
    "current_A": "100",
    "on_resistance_ohms": "0.1",
    "power_dissipation_W": "23"
}

get_mosfet(query, db)

/Users/narayanpowderly/Documents/atopile-workspace/.venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


{'Assembly Type': 'Wave SolderingA PCB assembly fixture is needed to protect and support this part during the assembly process.', 'Stock': 0, 'type': 'N Channel', 'Price': [{'quantity': '1+', 'price': 5.085}, {'quantity': '10+', 'price': 4.449}, {'quantity': '30+', 'price': 3.7035}, {'quantity': '90+', 'price': 3.321}, {'quantity': '450+', 'price': 3.1455}, {'quantity': '900+', 'price': 3.066}], 'Description': '500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS', 'CAD Model': 'PCB Footprint or Symbol', 'Datasheet': 'Download', 'current_A': '28', 'on_resistance_ohms': '0.155', 'Source': 'JLCPCB', 'JLCPCB Part #': 'C105685', 'Package': 'TO-3P-3', 'drain_source_voltage_V': '500', 'MFR.Part #': 'FDA28N50', 'footprint': '(module easyeda2kicad:TO-220IS_L10.16-W4.7-P2.54-L (layer F.Cu) (tedit 5DC5F6A4)\n\t(attr smd)\n\t(fp_text reference REF** (at 0 -4.0) (layer F.SilkS)\n\t\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\n\t(fp_text value TO-220IS_L10.16-W4.7-P2.54-L (at 0 4.0) (la

IndexError: list index out of range

In [ ]:
# Reference to the collection
mosfets_collection = db.collection('mosfets')

# Fetch documents
docs = mosfets_collection.stream()

for doc in docs:
    doc_data = doc.to_dict()

    # Convert numeric fields from string to actual numbers
    doc_data['Stock'] = int(doc_data['Stock'])
    doc_data['current_A'] = float(doc_data['current_A'])
    doc_data['on_resistance_ohms'] = float(doc_data['on_resistance_ohms'])
    doc_data['drain_source_voltage_V'] = float(doc_data['drain_source_voltage_V'])
    doc_data['power_dissipation_W'] = float(doc_data['power_dissipation_W'])
    for price in doc_data['Price']:
        price['price'] = float(price['price'])

    # Rename 'type' to 'polarity' and add new 'type'
    doc_data['polarity'] = doc_data.pop('type')
    doc_data['type'] = 'mosfet'

    # Update the document
    mosfets_collection.document(doc.id).set(doc_data)